In [47]:
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

import keras
from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [48]:
num_classes = 81
img_rows, img_cols = 32,32
batch_size = 64

train_data_dir = './Data/Training/'
validation_data_dir = './Data/Test/'

train_datagen = ImageDataGenerator(rescale = 1.0/255,
                                  rotation_range = 30,
                                  width_shift_range = .3,
                                  height_shift_range = .3,
                                  horizontal_flip = True,
                                  fill_mode = 'nearest')

validation_datagen = ImageDataGenerator(rescale = 1.0 / 255)

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                   target_size=(img_rows, img_cols),
                                                   batch_size = batch_size,
                                                   class_mode = 'categorical',
                                                   shuffle = True)

validation_generator = validation_datagen.flow_from_directory(validation_data_dir,
                                                              target_size = (img_rows, img_cols),
                                                              batch_size = batch_size,
                                                              class_mode = 'categorical',
                                                              shuffle = False)

Found 41322 images belonging to 81 classes.
Found 13877 images belonging to 81 classes.


In [49]:
input_shape = (img_rows, img_cols, 3)

model = Sequential()

model.add(Conv2D(32, (3,3), padding='same', input_shape=input_shape))
model.add(Activation('relu'))
model.add(Conv2D(32, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(.25))

model.add(Conv2D(64, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(.25))


model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_37 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
activation_55 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_56 (Activation)   (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_28 (Dropout)         (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 15, 15, 64)        18496     
__________

In [50]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.optimizers import RMSprop, SGD

checkpoint = ModelCheckpoint('./Model/fruits_fresh_cnn_1.h5',
                             monitor = 'val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss',
                         min_delta= 0,
                         patience = 3,
                         verbose=1,
                         restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                             factor=.2,
                             patience=3,
                             verbose = 1,
                             min_delta = .0001)

callbacks = [earlystop, checkpoint]

model.compile(loss='categorical_crossentropy',
             optimizer = RMSprop(lr=.001),
             metrics=['accuracy'])

no_train = 41322
no_val = 13877
epochs = 15

In [51]:
history = model.fit_generator(train_generator,
                    steps_per_epoch = no_train // batch_size,
                    epochs  = epochs,
                    callbacks = callbacks,
                    validation_data = validation_generator,
                    validation_steps=no_val // batch_size)

Epoch 1/15
645/645 [==============================] - 52s 80ms/step - loss: 2.2812 - acc: 0.3390 - val_loss: 0.6598 - val_acc: 0.8022

Epoch 00001: val_loss improved from inf to 0.65985, saving model to ./Model/fruits_fresh_cnn_1.h5
Epoch 2/15
645/645 [==============================] - 41s 63ms/step - loss: 0.8894 - acc: 0.7085 - val_loss: 0.7888 - val_acc: 0.7420

Epoch 00002: val_loss did not improve from 0.65985
Epoch 3/15
645/645 [==============================] - 39s 61ms/step - loss: 0.5533 - acc: 0.8218 - val_loss: 0.1623 - val_acc: 0.9416

Epoch 00003: val_loss improved from 0.65985 to 0.16229, saving model to ./Model/fruits_fresh_cnn_1.h5
Epoch 4/15
645/645 [==============================] - 39s 60ms/step - loss: 0.4094 - acc: 0.8669 - val_loss: 0.5556 - val_acc: 0.8381

Epoch 00004: val_loss did not improve from 0.16229
Epoch 5/15
645/645 [==============================] - 39s 60ms/step - loss: 0.3328 - acc: 0.8915 - val_loss: 0.1963 - val_acc: 0.9289

Epoch 00005: val_loss d